In [1]:
!git clone https://github.com/asellam/sitrex.git
%cd sitrex

fatal: destination path 'sitrex' already exists and is not an empty directory.
/content/sitrex


In [3]:
!pip install -r requirements.txt

ERROR: Operation cancelled by user


KeyboardInterrupt: 

In [2]:
import kagglehub

path = kagglehub.dataset_download('hasyimabdillah/workoutfitness-video')

In [9]:
import sys
sys.path.append('.')
# Import necessary libraries
import os
from sklearn.model_selection import train_test_split
from sitrex.preprocessing import calculate_angles
from sitrex.model import angle_usefulness_model, angle_similarity_model, TQDMProgressBar
import tensorflow as tf
import numpy as np

# EarlyStop save path for the Angle Similarity Transformer
MODEL_SAVE_PATH_USEFULNESS = './weights/angle_similarity_model_L300.h5'
# EarlyStop save path for the Angle Usefulness Transformer
MODEL_SAVE_PATH_SIMILARITY = './weights/angle_usefulness_model_L800.h5'
# Batch Size
batch_size = 32

# Load the trained Siamese model
similarity_model = angle_similarity_model(maxlen=300, module='transformer', lr=1e-4)
similarity_model.load_weights(MODEL_SAVE_PATH_SIMILARITY)
usefulness_model = angle_usefulness_model(maxlen=800, module='transformer', lr=1e-4)
usefulness_model.load_weights(MODEL_SAVE_PATH_USEFULNESS)
print("Models loaded successfully.")

Models loaded successfully.


In [4]:
import numpy as np

def normalize_landmarks(landmarks):
    if not landmarks or len(landmarks) < 33:
        return []

    # Body center calculation (using hips)
    left_hip = landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value]
    center = {
        'x': (left_hip.x + right_hip.x) / 2,
        'y': (left_hip.y + right_hip.y) / 2,
        'z': (left_hip.z + right_hip.z) / 2
    }

    # Scale calculation (shoulder width)
    left_shoulder = landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value]
    scale = abs(left_shoulder.x - right_shoulder.x) or 1e-5

    normalized = []
    for lm in landmarks:
        normalized.append({
            'x': (lm.x - center['x']) / scale,
            'y': (lm.y - center['y']) / scale,
            'z': lm.z - center['z'],
            'visibility': lm.visibility
        })
    return normalized

In [ ]:
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
from tqdm import tqdm

coach_video = 'videos/romanian deadlift/romanian deadlift_10.mp4'
athlete_video = 'videos/deadlift/deadlift_25.mp4'
poser = mp.solutions.pose.Pose(
            static_image_mode=False,
            model_complexity=2,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.7
        )

cap = cv2.VideoCapture(coach_video)
T = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
coach_angles = np.empty((T, 23), np.uint8)
for t in tqdm(range(T)):
    success, frame = cap.read()
    coach_frames[t, :, :, :] = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = poser.process(frame)
    landmarks = result.pose_landmarks.landmark
    angles = calculate_angles(normalize_landmarks(landmarks))
    names = sorted(list(angles.keys()))
    angles = [angles[name] for name in names]
    coach_angles[t, :] = angles
print("Total frames:", frame_count)

100%|████████████████████████████████████████████████| 140/140 [00:32<00:00,  4.29it/s]

Total frames: 140


In [ ]:
coach = np.empty((1, classifier_model.input_shape[1], classifier_model.input_shape[2]), np.float32)
if coach_angles.shape[0] < classifier_model.input_shape[1]:
    coach[0, :coach_angles.shape[0], :] = coach_angles[:, :]
    coach[0, coach_angles.shape[0]:, :] = 0
else:
    coach[0, :, :] = coach_angles[:classifier_model.input_shape[1], :]
usefulness = np.where(classifier_model.predict(coach)[0] < 0.5, 0, 1)
print(usefulness)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
[0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1]


In [ ]:
cap = cv2.VideoCapture(athlete_video)
T = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
FPS = int(cap.get(cv2.CAP_PROP_FPS))
athlete_frames = np.empty((T, H, W, 3), np.uint8)
athlete_angles = np.empty((T, 23), np.uint8)
athlete_landmarks = []
for t in tqdm(range(T)):
    success, frame = cap.read()
    if success:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = poser.process(frame)
        landmarks = result.pose_landmarks.landmark
        angles = calculate_angles(normalize_landmarks(landmarks))
        names = sorted(list(angles.keys()))
        angles = [angles[name] for name in names]
        athlete_frames[t, :, :, :] = frame
        athlete_angles[t, :] = angles
        athlete_landmarks.append(landmarks)

100%|████████████████████████████████████████████████| 938/938 [01:44<00:00,  8.95it/s]


In [ ]:
coach = np.empty((athlete_angles.shape[0], siamese_model.input_shape[0][1], siamese_model.input_shape[0][2]), np.float32)
if coach_angles.shape[0] < classifier_model.input_shape[1]:
    for t in range(athlete_angles.shape[0]):
        coach[t, :coach_angles.shape[0], :] = coach_angles[:, :]
        coach[t, coach_angles.shape[0]:, :] = 0
else:
    for t in range(athlete_angles.shape[0]):
        coach[t, :, :] = coach_angles[:siamese_model.input_shape[0][1], :]

athlete = np.empty((athlete_angles.shape[0], siamese_model.input_shape[1][1], siamese_model.input_shape[1][2]), np.float32)
for t in range(athlete_angles.shape[0]):
    if t < siamese_model.input_shape[1][1]:
        athlete[t, :t, :] = athlete_angles[:t, :]
        athlete[t, t:, :] = 0
    else:
        athlete[t, :, :] = athlete_angles[t-siamese_model.input_shape[1][1]:t, :]

similarity = siamese_model.predict([coach, athlete])

30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step


In [ ]:
from PIL import Image, ImageDraw
# Function to compute 23 specific angles from 33 keypoints
def get_angle_points(landmarks):
    """Compute 20 joint-angle features from Mediapipe landmarks."""
    if not landmarks or len(landmarks) < 33:
        return [0.0] * NUM_FEATURES
    # Define landmark indices
    LEFT_SHOULDER, RIGHT_SHOULDER = 11, 12
    LEFT_ELBOW, RIGHT_ELBOW = 13, 14
    LEFT_WRIST, RIGHT_WRIST = 15, 16
    LEFT_WRIST, RIGHT_WRIST = 15, 16
    LEFT_PINKY, RIGHT_PINKY = 17, 18
    LEFT_INDEX,RIGHT_INDEX = 19, 20
    LEFT_HIP, RIGHT_HIP = 23, 24
    LEFT_KNEE, RIGHT_KNEE = 25, 26
    LEFT_ANKLE, RIGHT_ANKLE = 27, 28

    # Estimating Primary Vectors
    L_shoulder = point(landmarks[LEFT_SHOULDER])  # LEFT_SHOULDER
    R_shoulder = point(landmarks[RIGHT_SHOULDER])  # RIGHT_SHOULDER
    L_hip = point(landmarks[LEFT_HIP])       # LEFT_HIP
    R_hip = point(landmarks[RIGHT_HIP])       # RIGHT_HIP
    L_ankle = point(landmarks[LEFT_ANKLE])       # LEFT_ANKLE
    R_ankle = point(landmarks[RIGHT_ANKLE])       # RIGHT_ANKLE

    # Midpoints
    mid_shoulder = midpoint(L_shoulder, R_shoulder)
    mid_hip = midpoint(L_hip, R_hip)

    # Local torso axes
    horizontal = normalize(R_shoulder - L_shoulder)           # Local X (shoulder width)
    upward = normalize(mid_shoulder - mid_hip)          # Local Y (torso up)
    forward = normalize(np.cross(horizontal, upward))          # Local Z (front-back normal to torso plane)
    # Re-orthogonalize Y just in case (to maintain right-handed frame)
    upward = normalize(np.cross(forward, horizontal))

    # Torso flexion
    mid_ankle = midpoint(L_ankle, R_ankle)
    torso = mid_shoulder - mid_hip
    gravity = mid_ankle - mid_hip
    torso_flexion = (mid_shoulder, mid_hip, mid_ankle)

    # Elbow
    left_elbow = (point(landmarks[LEFT_WRIST]), point(landmarks[LEFT_ELBOW]), point(landmarks[LEFT_SHOULDER]))
    right_elbow = (point(landmarks[RIGHT_WRIST]), point(landmarks[RIGHT_ELBOW]), point(landmarks[RIGHT_SHOULDER]))

    # Shoulder
    left_shoulder = (point(landmarks[LEFT_HIP]), point(landmarks[LEFT_SHOULDER]), point(landmarks[LEFT_ELBOW]))
    right_shoulder = (point(landmarks[RIGHT_HIP]), point(landmarks[RIGHT_SHOULDER]), point(landmarks[RIGHT_ELBOW]))

    # Wrist
    left_hand = midpoint(point(landmarks[LEFT_PINKY]), point(landmarks[LEFT_INDEX]))
    left_wrist = (left_hand, point(landmarks[LEFT_WRIST]), point(landmarks[LEFT_ELBOW]))
    right_hand = midpoint(point(landmarks[RIGHT_PINKY]), point(landmarks[RIGHT_INDEX]))
    right_wrist = (right_hand, point(landmarks[RIGHT_WRIST]), point(landmarks[RIGHT_ELBOW]))

    # Scapular upward rotation
    left_scapular_upward_rotation = (point(landmarks[LEFT_SHOULDER]), point(landmarks[LEFT_HIP]),  point(landmarks[RIGHT_HIP]))

    right_scapular_upward_rotation = (point(landmarks[RIGHT_SHOULDER]), point(landmarks[RIGHT_HIP]),  point(landmarks[LEFT_HIP]))

    # Shoulder abduction/extension
    shoulder = point(landmarks[LEFT_SHOULDER])
    v_arm_left = vector(landmarks[LEFT_SHOULDER],  landmarks[LEFT_ELBOW])
    f_proj_v_arm_left = vector_plane_project(v_arm_left, forward)
    f_left_shoulder_abduction = (shoulder + f_proj_v_arm_left, shoulder, shoulder + upward)
    h_proj_v_arm_left = vector_plane_project(v_arm_left, upward)
    h_left_shoulder_adduction = (shoulder + h_proj_v_arm_left, shoulder, shoulder + forward)
    s_proj_v_arm_left = vector_plane_project(v_arm_left, horizontal)
    left_shoulder_extension = (shoulder + s_proj_v_arm_left, shoulder, shoulder + upward)

    shoulder = point(landmarks[RIGHT_SHOULDER])
    v_arm_right = vector(landmarks[RIGHT_SHOULDER],  landmarks[RIGHT_ELBOW])
    f_proj_v_arm_right = vector_plane_project(v_arm_right, forward)
    f_right_shoulder_abduction = (shoulder + f_proj_v_arm_right, shoulder, shoulder + upward)
    h_proj_v_arm_right = vector_plane_project(v_arm_right, upward)
    h_right_shoulder_adduction = (shoulder + h_proj_v_arm_right, shoulder, shoulder + forward)
    s_proj_v_arm_right = vector_plane_project(v_arm_right, horizontal)
    right_shoulder_extension = (shoulder + s_proj_v_arm_right, shoulder, shoulder + upward)

    # Knee
    left_knee = (point(landmarks[LEFT_HIP]), point(landmarks[LEFT_KNEE]), point(landmarks[LEFT_ANKLE]))
    right_knee = (point(landmarks[RIGHT_HIP]), point(landmarks[RIGHT_KNEE]), point(landmarks[RIGHT_ANKLE]))

    # Hip
    left_hip = (point(landmarks[LEFT_SHOULDER]), point(landmarks[LEFT_HIP]), point(landmarks[LEFT_KNEE]))
    right_hip = (point(landmarks[RIGHT_SHOULDER]), point(landmarks[RIGHT_HIP]), point(landmarks[RIGHT_KNEE]))

    # Ankle dorsiflexion
    ankle = point(landmarks[LEFT_ANKLE])
    v_shin_left = vector(landmarks[LEFT_KNEE],  landmarks[LEFT_ANKLE])
    s_proj_v_shin_left = vector_plane_project(v_shin_left, horizontal)
    left_ankle_dorsiflexion = (ankle + s_proj_v_shin_left, ankle, ankle + upward)

    ankle = point(landmarks[RIGHT_ANKLE])
    v_shin_right = vector(landmarks[RIGHT_KNEE],  landmarks[RIGHT_ANKLE])
    s_proj_v_shin_right = vector_plane_project(v_shin_right, horizontal)
    right_ankle_dorsiflexion = (ankle + s_proj_v_shin_right, ankle, ankle + upward)

    # Left knee valgus indicator
    knee = point(landmarks[LEFT_KNEE])
    f_proj_left_hip = vector_plane_project(point(landmarks[LEFT_HIP]), forward)
    f_proj_left_knee = vector_plane_project(point(landmarks[LEFT_KNEE]), forward)
    f_proj_left_ankle = vector_plane_project(point(landmarks[LEFT_ANKLE]), forward)
    left_knee_valgus_indicator = (knee + f_proj_left_hip - f_proj_left_knee, knee, knee + f_proj_left_ankle - f_proj_left_knee)
    # Right knee valgus indicator
    knee = point(landmarks[RIGHT_KNEE])
    f_proj_right_hip = vector_plane_project(point(landmarks[RIGHT_HIP]), forward)
    f_proj_right_knee = vector_plane_project(point(landmarks[RIGHT_KNEE]), forward)
    f_proj_right_ankle = vector_plane_project(point(landmarks[RIGHT_ANKLE]), forward)
    right_knee_valgus_indicator = (knee + f_proj_right_hip - f_proj_right_knee, knee, knee + f_proj_right_ankle - f_proj_right_knee)

    angles = {
        'torso flexion': torso_flexion,
        'left elbow': left_elbow,
        'right elbow': right_elbow,
        'left shoulder': left_shoulder,
        'right shoulder': right_shoulder,
        'left wrist': left_wrist,
        'right wrist': right_wrist,
        'left frontal shoulder abduction': f_left_shoulder_abduction,
        'right frontal shoulder abduction': f_right_shoulder_abduction,
        'left scapular upward rotation': left_scapular_upward_rotation,
        'right scapular upward rotation': right_scapular_upward_rotation,
        'left knee': left_knee,
        'right knee': right_knee,
        'left hip': left_hip,
        'right hip': right_hip,
        'left ankle dorsiflexion': left_ankle_dorsiflexion,
        'right ankle dorsiflexion': right_ankle_dorsiflexion,
        'left knee valgus indicator': left_knee_valgus_indicator,
        'right knee valgus indicator': right_knee_valgus_indicator,
        'left horizontal shoulder adduction': h_left_shoulder_adduction,
        'right horizontal shoulder adduction': h_right_shoulder_adduction,
        'left shoulder extension': left_shoulder_extension,
        'right shoulder extension': right_shoulder_extension,
    }
    for key in angles:
        angles[key] = normalize_angle_points(angles[key])
    return angles

def standardize_landmarks(landmarks):
    if not landmarks or len(landmarks) < 33:
        return []
    normalized = []
    for lm in landmarks:
        normalized.append({
            'x': lm.x,
            'y': lm.y,
            'z': lm.z,
            'visibility': lm.visibility
        })
    return normalized

def normalize_angle_points(points):
    a, b, c = points
    la = np.linalg.norm(a - b)
    lc = np.linalg.norm(c - b)
    lm = max(la, lc)
    d = b + (a - b) * lm / la * 0.1
    e = b + (c - b) * lm / lc * 0.1
    a = b + (a - b) * lm / la * 0.3
    c = b + (c - b) * lm / lc * 0.3
    return b, a, c, d, e

def draw_pi_slice(draw, center, point1, point2, fill):
    radius = np.linalg.norm(point1 - center)
    bbox = [
        center[0] - radius, center[1] - radius,
        center[0] + radius, center[1] + radius
    ]

    def angle_between(p, center):
        dx = p[0] - center[0]
        dy = p[1] - center[1]
        angle = np.degrees(np.arctan2(dy, dx)) % 360
        return angle

    start_angle = angle_between(point1, center)
    end_angle = angle_between(point2, center)
    if (end_angle - start_angle) % 360 > 180:
        end_angle, start_angle = start_angle, end_angle
    draw.pieslice(bbox, start=start_angle, end=end_angle, fill=fill)

# Define the codec and create VideoWriter object
w, h = athlete_frames.shape[2], athlete_frames.shape[1]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID'
out = cv2.VideoWriter('output_video.mp4', fourcc, FPS, (w, h))

# Write images to video
for t in tqdm(range(athlete_frames.shape[0])):
    landmarks = athlete_landmarks[t]
    frame = Image.fromarray(athlete_frames[t])
    draw = ImageDraw.Draw(frame)
    angle_points = get_angle_points(standardize_landmarks(landmarks))
    for i in range(len(angle_points)):
        if usefulness[i] >= 0.5:
            a, b, c, d, e = angle_points[names[i]]
            r, g = int(255 * (1 - similarity[t, i])), int(255 * similarity[t, i])
            draw.line([a[0]*w, a[1]*h, b[0]*w, b[1]*h], fill=(r, g, 0), width=5)
            draw.line([a[0]*w, a[1]*h, c[0]*w, c[1]*h], fill=(r, g, 0), width=5)
            draw_pi_slice(draw,
                          np.array([a[0]*w, a[1]*h]),
                          np.array([d[0]*w, d[1]*h]),
                          np.array([e[0]*w, e[1]*h]),
                          (r, g, 0))
    out.write(cv2.cvtColor(np.array(frame), cv2.COLOR_BGR2RGB))  # Add frame

# Release everything
out.release()
print("Video saved as output_video.mp4")

100%|████████████████████████████████████████████████| 938/938 [00:20<00:00, 46.43it/s]

Video saved as output_video.mp4
